# Import packages

In [2]:
from textblob import TextBlob # sentiment analysis of textual data
import numpy as np # numeric and logical computation
import pandas as pd # data manipulation
import datetime # data type manipulation
from sklearn.model_selection import train_test_split # splitting data into train and test sets
from sklearn.linear_model import LinearRegression # LinearRegression model object
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score # evaluation metrics

# Import and clean data

In [3]:
# read in relevant columns of Kaggle dataset 
reviews = pd.read_excel('AirlineReviews.xlsx', 
                      header = 0, 
                      usecols = ['AirlineName', 'CabinType', 'DateFlown','EntertainmentRating', 'FoodRating', \
                                 'GroundServiceRating', 'OverallScore', 'Recommended', 'Review', \
                                 'SeatComfortRating', 'ServiceRating', 'TravelType', 'ValueRating', 'WifiRating', 'Route'])

# analyze shape of dataframe (num rows and columns)
print(reviews.shape)

# drop rows that have missing data
reviews = reviews.dropna(axis = 0)

# split route into origin/destination
reviews[['OriginCity', 'DestinationCity']] = reviews['Route'].str.split(' to ', expand = True)[[0,1]]
reviews['DestinationCity'] = reviews['DestinationCity'].str.split(' via ').str[0]
reviews.drop('Route', axis = 1)

# handle data types, column ordering, and row indexing
reviews['DateFlown'] = pd.to_datetime(reviews['DateFlown'])
reviews = reviews[['AirlineName', 'DateFlown', 'CabinType', 'TravelType', 'OriginCity', 'DestinationCity', \
                   'Recommended', 'Review', 'OverallScore', 'EntertainmentRating', 'FoodRating', \
                   'GroundServiceRating', 'SeatComfortRating', 'ServiceRating', 'ValueRating', 'WifiRating']]
reviews.reset_index(drop = True, inplace = True)

#reanalyze shape to ensure we have enough data for analysis
print(reviews.shape)

(129455, 15)
(89843, 16)


In [4]:
# get minimum and maximum dates to understand time period we are analyzing
print(reviews['DateFlown'].min())
print(reviews['DateFlown'].max())

2012-04-01 00:00:00
2023-05-01 00:00:00


# Add polarity columns representing review sentiment

In [5]:
# create empty dataframe to add sentiment information
sentiments = pd.DataFrame(columns = ['Polarity'])

#calculate polarity for each review and add value to sentiments dataframe
for index, row in reviews.iterrows():
    polarity = TextBlob(row['Review']).sentiment.polarity
    sentiments.loc[len(sentiments)] = [polarity]

In [6]:
# add sentiments dataframe to original reviews dataframe
reviews = pd.concat([reviews, sentiments], axis = 1)

In [7]:
# verify accurate concatentation of dataframes
reviews.tail()

,AirlineName,DateFlown,CabinType,TravelType,OriginCity,DestinationCity,Recommended,Review,OverallScore,EntertainmentRating,FoodRating,GroundServiceRating,SeatComfortRating,ServiceRating,ValueRating,WifiRating,Polarity
89838,Wizz Air,2015-05-01,Economy Class,Solo Leisure,Budapest,Barcelona,yes,"If you are going with them, you need to learn ...",8.0,0,0,2,1,1,5,0,-0.083333
89839,Wizz Air,2015-06-01,Economy Class,Family Leisure,Rome,Bucharest,no,We went to Ciampino airport 3 hours early and ...,1.0,1,1,1,1,2,1,1,0.082000
89840,Wizz Air,2015-04-01,Economy Class,Solo Leisure,London Luton,Budapest,no,I had never heard of Wizz Air before but as th...,1.0,0,0,2,2,1,2,0,0.100880
89841,Wizz Air,2015-06-01,Economy Class,Couple Leisure,Budapest,Tel Aviv,no,Very disappointing experience. The airline cha...,1.0,0,0,1,2,2,1,0,-0.006571
89842,Wizz Air,2015-02-01,Economy Class,Family Leisure,Budapest,Dubai,no,Wizz Air is hands down the worst airline in al...,1.0,0,1,1,1,1,1,0,-0.220062


In [8]:
# export cleaned dataframe with polarity to spreadsheet for further visualization
reviews.to_excel("reviewsPolarityClean.xlsx", index = False)

In [9]:
# get descriptive statistics for ratings and polarities
reviews.describe()

,OverallScore,EntertainmentRating,FoodRating,GroundServiceRating,SeatComfortRating,ServiceRating,ValueRating,WifiRating,Polarity
count,89843.000000,89843.000000,89843.000000,89843.000000,89843.000000,89843.000000,89843.000000,89843.000000,89843.000000
mean,3.988647,1.452701,1.879857,2.350756,2.459413,2.696359,2.471133,0.646784,0.065636
std,3.436031,1.719415,1.750159,1.636326,1.556247,1.712828,1.600606,1.238158,0.215369
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000
25%,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,-0.058520
50%,2.000000,1.000000,1.000000,1.000000,2.000000,3.000000,2.000000,0.000000,0.056250
75%,8.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,1.000000,0.197718
max,10.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.000000


# Analyze how ratings affect polarity

In [10]:
# get mean polarity for every available combination of rating
polarityByRatings = reviews.groupby(['EntertainmentRating', 'FoodRating', 'GroundServiceRating', 'SeatComfortRating', \
                                     'ServiceRating', 'ValueRating', 'WifiRating']).mean()

polarityByRatings

OverallScore  \
EntertainmentRating FoodRating GroundServiceRating SeatComfortRating ServiceRating ValueRating WifiRating                 
0                   0          0                   0                 0             0           0               1.125000   
                                                                                   1           0               1.095187   
                                                                                               1               1.250000   
                                                                                               2               1.000000   
                                                                                   2           0               1.531429   
...                                                                                                                 ...   
5                   5          5                   5                 5             5           1               9.750000   
                                                                                               2               9.700000   
                                                                                               3               9.672131   
                                                                                               4               9.720339   
                                                                                               5               9.871098   

                                                                                                           Polarity  
EntertainmentRating FoodRating GroundServiceRating SeatComfortRating ServiceRating ValueRating WifiRating            
0                   0          0                   0                 0             0           0           0.083895  
                                                                                   1           0          -0.009675  
                                                                                               1          -0.046561  
                                                                                               2           0.183333  
                                                                                   2           0           0.009697  
...                                                                                                             ...  
5                   5          5                   5                 5             5           1           0.336905  
                                                                                               2           0.301774  
                                                                                               3           0.336530  
                                                                                               4           0.365202  
                                                                                               5           0.354613  

[16189 rows x 2 columns]

### The cells below explore how an increase by one of any rating affects the polarity of reviews

In the first cell, wifi rating's relationship with polarity is explored. Loops iterate through every possible combination of the other six ratings. For every rating combination in the polarityByRatings dataframe, the mean polarity values for every wifi rating associated with that combination are isolated. The mean difference in polarity from one wifi rating to another with the given combination of other ratings is calculated and appended to the initalized list. Then the values in the list are averaged, handling for potential NaNs also appended by the for loop. The value printed is what an airline can expect the polarity of their review to increase by when the wifi rating increases by one. This is done for all seven ratings in the cells below. This can be used to interpret which aspects of an airline's product have had the largest impact on positive reviews from 2014 to 2023.


###### e = entertainment, f = food, gS = ground service, sC = seat comfortability, s = service, v = value, w = wifi

### Wifi

In [11]:
meanPolarityDiffsW = []

for e in range(6):
    for f in range(6):
        for gS in range(6):
            for sC in range(6):
                for s in range(6):
                    for v in range(6):
                        try:
                            meanPolarityDiffsW.append(
                                polarityByRatings.loc[pd.IndexSlice[e, f, gS, sC, s, v, :]].diff().mean()['Polarity'])
                        except:
                            pass
                        
print(np.mean(np.array(meanPolarityDiffsW)[np.logical_not(np.isnan(np.array(meanPolarityDiffsW)))]))

-0.004532878799570864


### Value

In [12]:
meanPolarityDiffsV = []

for e in range(6):
    for f in range(6):
        for gS in range(6):
            for sC in range(6):
                for s in range(6):
                    for w in range(6):
                        try:
                            meanPolarityDiffsV.append(
                                polarityByRatings.loc[pd.IndexSlice[e, f, gS, sC, s, :, w]].diff().mean()['Polarity'])
                        except:
                            pass
                        
print(np.mean(np.array(meanPolarityDiffsV)[np.logical_not(np.isnan(np.array(meanPolarityDiffsV)))]))

0.03371714502067232


### Service

In [13]:
meanPolarityDiffsS = []

for e in range(6):
    for f in range(6):
        for gS in range(6):
            for sC in range(6):
                for v in range(6):
                    for w in range(6):
                        try:
                            meanPolarityDiffsS.append(
                                polarityByRatings.loc[pd.IndexSlice[e, f, gS, sC, :, v, w]].diff().mean()['Polarity'])
                        except:
                            pass
                        
print(np.mean(np.array(meanPolarityDiffsS)[np.logical_not(np.isnan(np.array(meanPolarityDiffsS)))]))

0.023471429826453588


### Seat Comfortability

In [14]:
meanPolarityDiffsSC = []

for e in range(6):
    for f in range(6):
        for gS in range(6):
            for s in range(6):
                for v in range(6):
                    for w in range(6):
                        try:
                            meanPolarityDiffsSC.append(
                                polarityByRatings.loc[pd.IndexSlice[e, f, gS, :, s, v, w]].diff().mean()['Polarity'])
                        except:
                            pass
                        
print(np.mean(np.array(meanPolarityDiffsSC)[np.logical_not(np.isnan(np.array(meanPolarityDiffsSC)))]))

0.022765500254940153


### Ground Service

In [15]:
meanPolarityDiffsGS = []

for e in range(6):
    for f in range(6):
        for sC in range(6):
            for s in range(6):
                for v in range(6):
                    for w in range(6):
                        try:
                            meanPolarityDiffsGS.append(
                                polarityByRatings.loc[pd.IndexSlice[e, f, :, sC, s, v, w]].diff().mean()['Polarity'])
                        except:
                            pass
                        
print(np.mean(np.array(meanPolarityDiffsGS)[np.logical_not(np.isnan(np.array(meanPolarityDiffsGS)))]))

0.026840621223430366


### Food

In [16]:
meanPolarityDiffsF = []

for e in range(6):
    for gS in range(6):
        for sC in range(6):
            for s in range(6):
                for v in range(6):
                    for w in range(6):
                        try:
                            meanPolarityDiffsF.append(
                                polarityByRatings.loc[pd.IndexSlice[e, :, gS, sC, s, v, w]].diff().mean()['Polarity'])
                        except:
                            pass
                        
print(np.mean(np.array(meanPolarityDiffsF)[np.logical_not(np.isnan(np.array(meanPolarityDiffsF)))]))

0.01680920391806202


### Entertainment

In [17]:
meanPolarityDiffsE = []

for f in range(6):
    for gS in range(6):
        for sC in range(6):
            for s in range(6):
                for v in range(6):
                    for w in range(6):
                        try:
                            meanPolarityDiffsE.append(
                                polarityByRatings.loc[pd.IndexSlice[:, f, gS, sC, s, v, w]].diff().mean()['Polarity'])
                        except:
                            pass
                        
print(np.mean(np.array(meanPolarityDiffsE)[np.logical_not(np.isnan(np.array(meanPolarityDiffsE)))]))

0.002653060056288826


# Construct linear regression model to predict overall score

In [18]:
# create new month flown variable and one-hot encode to capture seasonality
reviews['MonthFlown'] = [each.month for each in reviews['DateFlown']]

# add categorical one hot encoded values to original reviews dataframe
reviews = pd.concat([reviews, \
                    pd.get_dummies(reviews['MonthFlown'], "month", drop_first = True), \
                    pd.get_dummies(reviews['CabinType'], "cabin", drop_first = True), \
                    pd.get_dummies(reviews['TravelType'], "travel", drop_first = True), \
                    pd.get_dummies(reviews['Recommended'], "rec", drop_first = True)], axis = 1)


# drop columns that were one hot encoded
reviews = reviews.drop(['MonthFlown', 'CabinType', 'TravelType', 'Recommended'], axis = 1)

# display reviews with one hot encoded columns
reviews.columns

Index(['AirlineName', 'DateFlown', 'OriginCity', 'DestinationCity', 'Review',
       'OverallScore', 'EntertainmentRating', 'FoodRating',
       'GroundServiceRating', 'SeatComfortRating', 'ServiceRating',
       'ValueRating', 'WifiRating', 'Polarity', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'month_12', 'cabin_Economy Class',
       'cabin_First Class', 'cabin_Premium Economy', 'travel_Couple Leisure',
       'travel_Family Leisure', 'travel_Solo Leisure', 'rec_yes'],
      dtype='object')

In [20]:
# create X and y dataframes for model
X = reviews.drop(['AirlineName', 'DateFlown', 'OriginCity', 'DestinationCity', 'Review', 'OverallScore', 'Polarity'], axis = 1)
y = reviews['OverallScore'].astype(int)

In [21]:
# explore independent variables
X.head()

,EntertainmentRating,FoodRating,GroundServiceRating,SeatComfortRating,ServiceRating,ValueRating,WifiRating,month_2,month_3,month_4,...,month_10,month_11,month_12,cabin_Economy Class,cabin_First Class,cabin_Premium Economy,travel_Couple Leisure,travel_Family Leisure,travel_Solo Leisure,rec_yes
0,0,4,4,4,5,3,0,0,0,0,...,0,1,0,1,0,0,0,0,1,1
1,0,1,1,2,2,2,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,1,1,2,1,2,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,1,1,1,3,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,0,0,5,4,4,4,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1


In [22]:
# explore target variable
y[:5]

0    9
1    1
2    1
3    2
4    9
Name: OverallScore, dtype: int32

In [23]:
# create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 10)

# ensure we have enough data to test
print(X_train.shape)
print(X_test.shape)

# initialize model
model = LinearRegression()

# fit model on training set
model.fit(X_train, y_train)

# predict on test set
predictions = model.predict(X_test)

# model evaluation
print('r2 score : ', r2_score(y_test, predictions))
print('mean squared error : ', mean_squared_error(y_test, predictions))
print('mean absolute error : ', mean_absolute_error(y_test, predictions))

(80858, 25)
(8985, 25)
r2 score :  0.9159397767381177
mean squared error :  0.9837983769917354
mean absolute error :  0.6941952235465192


In [24]:
pd.DataFrame(zip(X.columns, model.coef_), columns = ['Independent Variable', 'Coefficient'])\
.sort_values('Coefficient', ascending = False).reset_index(drop = True)

,Independent Variable,Coefficient
0,rec_yes,2.911231
1,ValueRating,0.649117
2,GroundServiceRating,0.352550
3,SeatComfortRating,0.130131
4,ServiceRating,0.120909
5,FoodRating,0.094079
6,travel_Solo Leisure,0.028085
7,month_2,0.020466
8,month_10,0.020295
9,month_9,0.017814
